In [2]:
import pickle
import numpy as np
from tqdm import tqdm

In [10]:
def rad_to_deg(rad):
    return rad * 180 / np.pi

In [4]:
connected_joints = [(0, 1),(0, 16),(0, 12),(1, 20),(20, 2),(2, 3),(20, 8),(20, 4),(8, 9),(9, 10),(10, 11),(11, 23),(11, 24),(4, 5),(5, 6),(6, 7),(7, 21),(7, 22),(16, 17),(17, 18),(18, 19),(12, 13),(13, 14),(14, 15)]

connected_joints = [(x-1, y-1) for x, y in connected_joints]

In [5]:
joints = {}
X_path = 'C:\\Users\\tcarr23\\Downloads\\X.pkl'
try:
    with open('l2-norm-dict.pkl', 'rb') as f:
        joints = pickle.load(f)
except:
    with (open(X_path, "rb")) as f:
        X = pickle.load(f)

    for j in connected_joints:
        joints[f"{(int(j[0]) + 1)}-{(int(j[1]) + 1)}"] = []

    for p in tqdm(X, desc="Actor", position=0):
        for frame in tqdm(X[p], desc="Frame", position=1, leave=False, miniters=int(len(X[p])/10)):
            for j in connected_joints:
                joints[f"{(int(j[0]) + 1)}-{(int(j[1]) + 1)}"].append(np.linalg.norm(np.array([frame[j[0] * 3], frame[j[0] * 3 + 1], frame[j[0] * 3 + 2]]) - np.array([frame[j[1] * 3], frame[j[1] * 3 + 1], frame[j[1] * 3 + 2]])))
    
    with open('l2-norm-dict.pkl', 'wb') as f:
        pickle.dump(joints, f)

    average_skeleton_joints = {}
    for j in joints:
        average_skeleton_joints[j] = np.mean(joints[j])

    with open('average-skeleton-joints.pkl', 'wb') as f:
        pickle.dump(average_skeleton_joints, f)

Actor: 100%|██████████| 106/106 [14:11<00:00,  8.03s/it]


In [3]:
with open('average-skeleton-joints.pkl', 'rb') as f:
    average_skeleton_joints = pickle.load(f)
average_skeleton_joints

{'0-1': 0.4333,
 '0-16': 0.925,
 '0-12': 0.04556,
 '1-20': 0.6714,
 '20-2': 0.928,
 '2-3': 0.285,
 '20-8': 0.877,
 '20-4': 1.305,
 '8-9': 0.4976,
 '9-10': 0.2344,
 '10-11': 0.2286,
 '11-23': 0.391,
 '11-24': 0.1262,
 '4-5': 0.2756,
 '5-6': 0.2367,
 '6-7': 0.229,
 '7-21': 0.425,
 '7-22': 0.1247,
 '16-17': 0.685,
 '17-18': 0.3394,
 '18-19': 0.3533,
 '12-13': 0.4375,
 '13-14': 0.3394,
 '14-15': 0.353}

In [6]:
test = X['P001'][0]
test

array([ 0.2181,  0.1726,  3.785 ,  0.2323,  0.4326,  3.715 ,  0.2457,
        0.6875,  3.635 ,  0.2129,  0.808 ,  3.582 ,  0.1109,  0.6113,
        3.717 ,  0.1009,  0.4287,  3.742 ,  0.1187,  0.3428,  3.557 ,
        0.1165,  0.3174,  3.55  ,  0.345 ,  0.574 ,  3.588 ,  0.4211,
        0.3848,  3.627 ,  0.1919,  0.3188,  3.541 ,  0.1299,  0.3171,
        3.543 ,  0.1627,  0.1746,  3.78  ,  0.1975, -0.1361,  3.904 ,
        0.2443, -0.4202,  4.055 ,  0.204 , -0.4763,  4.094 ,  0.27  ,
        0.1678,  3.73  ,  0.3105, -0.1462,  3.818 ,  0.3428, -0.433 ,
        3.943 ,  0.2952, -0.502 ,  3.969 ,  0.2426,  0.6245,  3.656 ,
        0.12  ,  0.2957,  3.533 ,  0.099 ,  0.346 ,  3.527 ,  0.0871,
        0.2988,  3.547 ,  0.1268,  0.2815,  3.549 ], dtype=float16)

In [7]:
end_effectors = [
    3, # Head
    23, # Tip fo Right Hand
    24, # Right Thumb
    21, # Tip of Left Hand
    22, # Left Thumb
    15, # Left Foot
    19, # Right Foot
]

In [8]:
paths_to_end_effectors = [
    [0,1,20,2,3],
    [0,1,20,8,9,10,11,23],
    [0,1,20,8,9,10,11,24],
    [0,1,20,4,5,6,7,21],
    [0,1,20,4,5,6,7,22],
    [0,12,13,14,15],
    [0,16,17,18,19]
]

In [9]:
for path in paths_to_end_effectors:
    for connection in range(len(path) - 1):
        vector = test[path[connection] * 3: path[connection] * 3 + 3] - test[path[connection + 1] * 3: path[connection + 1] * 3 + 3]
        length = np.linalg.norm(vector)
        normalized_vector = vector / length
        print(f"{path[connection]}-{path[connection + 1]}: {length} - {normalized_vector}")

        x_rotation = np.arctan2(normalized_vector[1], normalized_vector[2])
        y_rotation = np.arctan2(normalized_vector[0], normalized_vector[2])
        z_rotation = np.arctan2(normalized_vector[0], normalized_vector[1])

        print(f"Radians: X: {x_rotation} Y: {y_rotation} Z: {z_rotation}")
        print(f"Degrees: X: {rad_to_deg(x_rotation)} Y: {rad_to_deg(y_rotation)} Z: {rad_to_deg(z_rotation)}")

0-1: 0.269775390625 - [-0.0525 -0.964   0.2607]
X: -1.306640625 Y: -0.1986083984375 Z: -3.087890625
1-20: 0.200927734375 - [-0.05103 -0.955    0.2915 ]
X: -1.2744140625 Y: -0.17333984375 Z: -3.087890625
20-2: 0.066650390625 - [-0.0476 -0.945   0.3223]
X: -1.2421875 Y: -0.1466064453125 Z: -3.091796875
2-3: 0.1356201171875 - [ 0.2421 -0.889   0.389 ]
X: -1.158203125 Y: 0.556640625 Z: 2.875
0-1: 0.269775390625 - [-0.0525 -0.964   0.2607]
X: -1.306640625 Y: -0.1986083984375 Z: -3.087890625
1-20: 0.200927734375 - [-0.05103 -0.955    0.2915 ]
X: -1.2744140625 Y: -0.17333984375 Z: -3.087890625
20-8: 0.1329345703125 - [-0.7705  0.3784  0.514 ]
X: 0.63427734375 Y: -0.982421875 Z: -1.1142578125
8-9: 0.2078857421875 - [-0.3665  0.911  -0.1879]
X: 1.7744140625 Y: -2.044921875 Z: -0.38232421875
9-10: 0.25341796875 - [0.905 0.26  0.339]
X: 0.654296875 Y: 1.2119140625 Z: 1.291015625
10-11: 0.06207275390625 - [ 0.999    0.02753 -0.03146]
X: 2.421875 Y: 1.6025390625 Z: 1.54296875
11-23: 0.0467224121093